### Introduction: Applying Computational Thinking to Personal Finance and Portfolio Analysis
In an era where financial independence is closely tied to the ability to make strategic investment decisions, understanding the mechanics of saving and investing has become a critical life skill. Yet, while many individuals understand the importance of saving, few have the tools or knowledge required to evaluate how different financial strategies—and especially investment decisions—impact long-term outcomes. This tutorial addresses that gap by introducing students to essential concepts in personal finance and portfolio analysis through a computational and data-driven approach.

This tutorial begins by building a foundation in core savings principles. Learners will explore the functionality of common financial instruments such as high-yield savings accounts, 401(k)s, and Roth IRAs. Through interactive inputs and simulations, students will see how interest rates, compound growth, and tax advantages affect returns over time. These early exercises aim to demonstrate the value of starting early, contributing consistently, and choosing the right savings vehicles for one’s personal financial goals.

Building on this foundation, the tutorial then transitions into a more advanced, investment-focused module: constructing and evaluating a personalized stock portfolio. Students will be guided through the process of:
- Selecting their own breakdown of stocks to form a diversified portfolio based on individual risk preferences or interests.

- Evaluating the strength of their chosen portfolio using historical data and key financial metrics.

- Simulating potential future outcomes through Monte Carlo simulations, which model thousands of possible future scenarios to account for uncertainty and market volatility.

- Optimizing the stock allocation using algorithmic methods that balance return potential with risk, aiming for the most efficient use of investment capital.

- Rechecking the portfolio’s strength after optimization, to reflect the improvements made and demonstrate the impact of computational adjustments on financial performance.


Throughout this tutorial, students will employ principles from computer science—such as data modeling, simulations, algorithmic thinking, and data visualization—to analyze financial data and make informed decisions. By the end of the tutorial, students will have developed not only a deeper understanding of financial literacy but also a practical appreciation for how computational tools can empower individuals to manage their personal wealth more effectively.

This interdisciplinary exploration is designed to equip learners with a strong foundation in both personal finance and applied computation—skills that are increasingly essential in today’s data-rich, financially complex world.

### Disclaimer

**No Guarantee of Accuracy:** While Isabel, Luna, and Nirantheri strive to provide accurate and up-to-date information, they do not guarantee the accuracy, completeness, or reliability of any content. Users should independently verify any information before making financial or investment decisions based on it.


**Investment Risks:** Investing involves inherent risks, including but not limited to market fluctuations, economic uncertainty, geopolitical events, and individual asset performance. Past performance is not indicative of future results, and no content provided implies a guarantee of investment success.

**Limitation of Liability:** Mentions of specific financial products, services, companies, or securities within the content do not constitute endorsements or recommendations. Users are responsible for conducting their own research and due diligence.

**No Liability:** Isabel, Luna, Nirantheri, and their affiliates, partners, or contributors shall not be held liable for any loss, damage, or expense resulting from the use of or reliance on the provided information. All investment decisions made based on this content are the sole responsibility of the user.

**Disclaimer Updates:** This disclaimer is subject to change without notice. Users are responsible for reviewing it periodically to stay informed of any updates.



### using old material : Nirantheri


The code to show the importance of investing vs savings (just draw out each of the ira, 401k, hysa along with no investment to show)
How to use information.md?

### setup

include all packages for setting up (pyfolio, package)

In [ ]:
# all libraries

# import pyfolio
import yfinance as yf
import numpy as np
import scipy
import ipywidgets as widgets
import pandas as pd
from scipy.optimize import minimize
from IPython.display import display
import altair as alt
from pandas_datareader import data as wb
import seaborn as sns
from scipy.stats import norm

import matplotlib.pyplot as plt
%matplotlib inline

### Choosing stocks for a portfolio: Nirantheri


Apple: 'AAPL'
Costco: 'COST'
Microsoft: 'MSFT'
Google: 'GOOG'
Nvidia: 'NVDA'
Walmart: 'WMT'
AMC Entertainment 'AMC'
S&P 500: '^GSPC'
Dow Jones Industrial: '^DJI'
Nasdaq: '^IXIC'
Vanguard S&P 500 ETF: 'VOO'
Vanguard Total Stock Market Index Fund: 'VSMPX'
Fidelity 500 Index Fund: 'FXAIX'
Gold: 'GLD'

Give them a list of 20-30 common stocks, etfs, and mutual funds (can use checkbox widget)
ipywidgets: 
[checkbox](https://ipywidgets.readthedocs.io/en/7.x/examples/Widget%20List.html#Checkbox)
[select multiple](https://ipywidgets.readthedocs.io/en/7.x/examples/Widget%20List.html#SelectMultiple)
ex.
- S&P 500
- Dow
- Nasdaq
- Apple
- Tesla
- Google
- Gold??
- etc

In [ ]:
codes = {"Apple": 'AAPL',
        "Costco": 'COST',
        "Microsoft": 'MSFT',
        "Google": 'GOOG',
        "Nvidia": 'NVDA',
        "Walmart": 'WMT',
        "AMC Entertainment": 'AMC',
        "S&P 500": '^GSPC',
        "Dow Jones Industrial": '^DJI',
        "Nasdaq": '^IXIC',
        "Vanguard S&P 500 ETF": 'VOO',
        "Fidelity 500 Index Fund": 'FXAIX',
        "Gold":'GLD'}

codes.keys()


choices = widgets.SelectMultiple(
    options=codes.keys(),
    value=[],
    rows=20,
    description='Stocks',
    disabled=False
)

display(choices)

# isabel's code may work better here.


### Picking your own breakdown of stocks : Nirantheri

use another widget, maybe with a slider (tbd)

[setup using the above code chunk](https://ipywidgets.readthedocs.io/en/7.x/examples/Widget%20List.html#Tabs)

In [ ]:
# get breakdown

# get codes for further down
portfolio = []

for i in range(len(choices.value)):
    portfolio.append(codes[choices.value[i]])
    
# pick distribution

tab_contents = ["Value"] * len(choices.value)
children = [widgets.Text(description=name) for name in tab_contents]
tab = widgets.Tab()
tab.children = children
tab.titles = choices.value

tab


In [ ]:
# matching allocation amounts to stock codes given inputs from above

allocations = {}

for i in range(len(tab.children)):
    allocations[portfolio[i]]=tab.children[i].value




### pulling the yf data: Luna

Assume that the list of stocks will come in as a set of stock codes so you can write a loop to get each of them

In [ ]:
#pulled a dictionary of different stocks, keys are tickers? i cant think of a better way to store them because the list is variable so its a bit of an issue

# testList = ['AAPL', 'MSFT', 'GOOG', 'NVDA', '^GSPC', '^DJI', '^IXIC']
if len(portfolio)==0: # for testing without ipywidget
    portfolio = ['AAPL', 'MSFT',  '^IXIC']
    allocations={'AAPL':'0.4', 'MSFT':'0.3',  '^IXIC':'0.3'}

testList=portfolio
testString = ' '.join(testList)

tickersPull = yf.Tickers(testString)

#in the dictionary dataframes, each ticker is indexed by its ticker as seen in the testList (auto_adjust accounts for splits and dividends)
dataframes = {}
for x in testList:
    ticker = yf.Ticker(x)
    dataframes[x] = ticker.history(period='1mo', start='2015-01-01', auto_adjust=True)
    # print(dataframes[x])



## Checking the returns with this : Nirantheri

Using the yf data, we will check what returns looked like over time.

#### TODO: should we try to project or work with previous data??

https://blog.mlq.ai/python-for-finance-portfolio-optimization/


## How do we calculate returns?

To calculate the returns, we want to find the adjusted value of each stock amount from the date of the initial investment. We can write a function to do this with a set of inputs-- an initial portfolio value, the set of dataframes for each stock, and the breakdown of allocations  <!-- and perhaps even an initial date of investment.  -->

In [ ]:
def calculate_returns(initial_portfolio_val, dataframe, allocations):
    all_pos_vals = []

    for stock_name in dataframe:
        # grab the dataframe for a single stock
        stock_df = dataframe[stock_name] 
        # create normed return column
        stock_df['Normed Return'] = stock_df['Close'] /stock_df.iloc[0]['Close']

        # use normed return to adjust the percentage of portfolio held
        allocation = float(allocations[stock_name])  # Convert allocation to float
        stock_df['Allocation'] = stock_df['Normed Return']*allocation

        # find value of stock at each date
        stock_df['Position Value'] = stock_df['Allocation']*initial_portfolio_val

        # add to list of all position values
        all_pos_vals.append(stock_df['Position Value'])


    # concatenate the list of position values
    portfolio_val = pd.concat(all_pos_vals, axis=1)

    # set the column names
    portfolio_val.columns = portfolio

    # add a total portfolio column
    portfolio_val['Total'] = portfolio_val.sum(axis=1)

    # changing date to column not index
    portfolio_val = portfolio_val.reset_index()
    
    return portfolio_val

Then we can create a graphical representation of each of the stocks' growths over time as well as the portfolio's overall growth. To get an overall number rather than using every single data point, we can use a resampling function to grab the last day of each month in the data.

In [ ]:
def graph_values(portfolio_values, graph_portfolio, graph_stocks):
    """graph_portfolio is a bool to show only the total portfolio, graph_stocks is to graph all stocks."""

    # Resample the portfolio_val DataFrame to only include the last date of each month
    portfolio_val = portfolio_values.resample('ME', on='Date').last().reset_index()

    if graph_portfolio:
        portfolio_val['Total'].plot(figsize=(10,8))
    if graph_stocks:
        portfolio_val.drop('Total', axis=1).set_index('Date').plot(figsize=(10,8)) 


    print("total value", portfolio_val['Total'].iat[-1])


Now that we've created a projection of how our data has been growing over time, let's see what to expect!

In [ ]:
returns = calculate_returns(1e6, dataframes, allocations)

graph_values(returns, True, True)

In [ ]:
# TODO: Implement the altair version which has more interactivity

# # need to pivot the data
# # print(portfolio_val.head())
# print(portfolio_val.info())
# portfolio_val.drop("^IXIC", axis=1)
# pivoted_data = portfolio_val.melt(id_vars="Date", var_name="Stock", value_name="Value")

# pivoted_data.head()


# alt.Chart(pivoted_data).mark_line().encode(
#     alt.X("Date:T", title="Date"),
#     alt.Y("Value:Q", title="Portfolio Value"),
#     alt.Color("Stock:N", title="Stock")
# )


# plot our portfolio

### check strength of portfolio : Isabel
pyfolio: https://www.pyquantnews.com/the-pyquant-newsletter/create-beautiful-strategy-tear-sheets-pyfolio-reloaded
can also use quantstats



pyfolio: get the tear sheets--> then use pandas to pull specific metrics (annual return, etc) and then have a md cell which breaks down what each of them means

In [ ]:
##Isabel's Interactive Stock Picker + Allocation Setup

import ipywidgets as widgets
from IPython.display import display, clear_output

# Available stock options
codes = {
    "Apple": 'AAPL',
    "Microsoft": 'MSFT',
    "Google": 'GOOG',
    "Nvidia": 'NVDA',
    "Tesla": 'TSLA',
    "S&P 500": '^GSPC',
    "Dow Jones": '^DJI',
    "Nasdaq": '^IXIC',
    "Gold": 'GLD'
}

# Stock selection widget
stock_selector = widgets.SelectMultiple(
    options=codes.keys(),
    value=[],
    rows=10,
    description='Stocks',
    disabled=False
)

# Button to confirm selection and show allocation inputs
submit_btn = widgets.Button(description="Submit Stock Selection")

# Output area to update UI
output = widgets.Output()

# Callback for the button
def on_submit_clicked(b):
    output.clear_output()
    with output:
        selected = list(stock_selector.value)
        if not selected:
            print("❌ Please select at least one stock.")
            return

        allocation_inputs = []
        print("📊 Enter your allocation percentages (e.g. 0.3 for 30%)")
        for stock in selected:
            input_widget = widgets.Text(
                description=stock,
                placeholder="Enter % (0 to 1)"
            )
            allocation_inputs.append((stock, input_widget))
            display(input_widget)

        # Save allocations on second button press
        def finalize_allocations(btn):
            global allocations
            allocations = {}
            try:
                for stock, widget in allocation_inputs:
                    val = float(widget.value)
                    if val < 0 or val > 1:
                        raise ValueError("Out of bounds")
                    allocations[codes[stock]] = val
                print("\n✅ Allocations saved!")
                print("Portfolio:", [codes[stock] for stock in selected])
                print("Allocations:", allocations)
            except Exception as e:
                print("⚠️ Error: Make sure all values are valid numbers between 0 and 1.")

        confirm_btn = widgets.Button(description="Confirm Allocations")
        confirm_btn.on_click(finalize_allocations)
        display(confirm_btn)

submit_btn.on_click(on_submit_clicked)

display(stock_selector, submit_btn, output)


### montecarlo simulations : tbd

uses yahoo finance package and others to create montecarlo simulation for modeling

montecarlo https://medium.com/analytics-vidhya/monte-carlo-simulations-for-predicting-stock-prices-python-a64f53585662

In [74]:
df = pd.DataFrame()
ticker = yf.Ticker("GOOG")
df['GOOG'] = ticker.history(interval = '1d', start = '2015-01-01')['Close']

log_return = np.log(1 + df.pct_change().dropna())

print(log_return)

# sns.displot(log_return)
# plt.xlabel("Daily Return")
# plt.ylabel("Frequency")

u = log_return.mean()
var = log_return.var()
drift = u - (.5*var)

stdev = log_return.std()
days = 50
trials = 10000
Z = norm.ppf(np.random.rand(days, trials))
daily_return = np.exp(drift.values + stdev.values * Z)

price_paths = np.zeros_like(daily_return)
price_paths[0] = df.iloc[-1]
for t in range(1, days):
    price_paths[t] = price_paths[t-1]*daily_return[t]

print(price_paths)

                               GOOG
Date                               
2015-01-05 00:00:00-05:00 -0.021066
2015-01-06 00:00:00-05:00 -0.023450
2015-01-07 00:00:00-05:00 -0.001715
2015-01-08 00:00:00-05:00  0.003148
2015-01-09 00:00:00-05:00 -0.013035
...                             ...
2025-04-16 00:00:00-04:00 -0.020244
2025-04-17 00:00:00-04:00 -0.013858
2025-04-21 00:00:00-04:00 -0.023087
2025-04-22 00:00:00-04:00  0.026601
2025-04-23 00:00:00-04:00  0.024518

[2591 rows x 1 columns]
[[157.72000122 157.72000122 157.72000122 ... 157.72000122 157.72000122
  157.72000122]
 [157.58357925 155.17542685 153.51091667 ... 159.04733528 158.08754554
  160.28472906]
 [153.74154916 153.66652609 152.95922949 ... 154.0846565  159.56948671
  160.49654942]
 ...
 [165.68519622 141.4075013  145.43770903 ... 127.92932855 125.96587148
  144.79242235]
 [164.37759019 142.97603074 141.53780248 ... 131.21568518 130.40560114
  143.80717961]
 [167.35055118 141.18456973 139.24293296 ... 129.78189906 130.59939

In [88]:
def import_stock_data(tickers, start = "2015-01-01"):
    data = pd.DataFrame()
    if len(tickers) == 1:
        stock = yf.Ticker(tickers[0])
        data[tickers[0]] = stock.history(interval = '1d', start = start)['Close']
    else:
        for x in tickers:
            stock = yf.Ticker(x)
            data[x] = stock.history(interval = '1d', start = start)['Close']
    return data

def log_returns(data): 
    return np.log(1 + data.pct_change().dropna())

def simple_returns(data):
    return (data/(data.shift(1))-1)

def market_data_combination(data, mark_ticker='^GSPC', start = '2015-01-01'):
    market_data = import_stock_data(mark_ticker, start)
    market_rets = log_returns(market_data)
    ann_return = np.exp(market_rets.mean() * 252).values-1
    data = data.merge(market_data, left_index=True, right_index=True)
    return data, ann_return

def drift_calc(data, return_type = "log"): 
    if return_type == "log":
        lr = log_returns(data)
    elif return_type == "simple":
        lr = simple_returns(data)
    u = lr.mean()
    var = lr.var()
    drift = u-(.5*var)
    try: 
        return drift.values
    except:
        return drift

def daily_returns(data, days, iterations):
    ft = drift_calc(data)
    try: 
        stv = log_returns(data).std().values
    except:
        stv = log_returns(data).std()

    dr = np.exp(ft + stv * norm.ppf(np.random.rand(days, iterations)))
    return dr

def simulate_mc(data, days, iterations, return_type='log', plot=True):
    returns = daily_returns(data, days, iterations)
    price_list = np.zeros_like(returns)
    price_list[0] = data.iloc[-1]
    for t in range(1, days):
        price_list[t] = price_list[t-1] * returns[t]
    
    # if plot:
    #     x = pd.DataFrame(price_list).iloc[-1]
    #     fig, ax = plt.subplots(1,2, figsize=(14,4))
    #     sns.displot(x, ax=ax[0])
    #     sns.displot(x, hist_kws={'cumulative':True},kde_kws={'cumulative':True}, ax=ax[1])
    #     plt.xlabel('Stock Price')
    #     plt.show()
    

    total_days = days - 1
    expected_value = round(pd.DataFrame(price_list).iloc[-1].mean(), 2)
    return_pct = round(100*(expected_value-price_list[0,1])/expected_value, 2)

    return [total_days, expected_value, return_pct]

def monte_carlo(ticker, days, iterations):
    df = import_stock_data([ticker])
    return simulate_mc(df, days, iterations)

print(monte_carlo('GOOG', 10, 100))

[9, 163.04, 0.96]


### optimizing the set of stocks : luna

use scipy

optimizing your investment: https://medium.com/@ethan.duong1120/python-powered-portfolio-optimization-achieving-target-returns-through-weight-optimization-fc5163e5c9c6


It's great that we can create a stock portfolio that we can work with, and we can even analyze how strong it is using pyfolio! It would be even better if we could take the portfolio we made and see if we can change our investment amounts to maximize our returns. There are a couple steps to get it set up, but thankfully, scipy has a function that can maximize our returns given an initial investment percentage for each stock, and a history of each stock's percentage change in price.

First, we are going to create a sample version of our portfolio that pulls stock history for every ticker that we have selected, adding their opening price at every interval to the function and storing all of this in a dataframe. Then, we use the pct_change function to determine the change between each frame for each stock, allowing us to determine the most efficient investment of stocks.

In [ ]:
# testList = ['AAPL', 'MSFT', 'GOOG', 'TSLA', 'NFLX', 'SBUX', '^GSPC', '^DJI', '^IXIC']

testList = portfolio

# pull the first stock's data; drop all unnecessary columns
ticker = yf.Ticker(testList[0])
df = ticker.history(interval = '1d', start = '2015-01-01', end = '2025-04-01')
df.drop(columns=['High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'], inplace=True)
df.rename({'Open' : 'AAPL'}, inplace=True)

# for every other ticker, add its open value to the existing dataframe
for x in testList[1:]:
    ticker = yf.Ticker(x)
    data = ticker.history(interval = '1d', start = '2015-01-01', end = '2025-04-01')
    df.insert(len(df.columns), x, data["Open"]) # for whatever reason, this line wont work

#calculate percent returns for each day of each stock
returns_df = df.pct_change(1).dropna()

Now, we need to determine the value of each portfolio's return. This is simply the return of each stock multipled by its weight in the portfolio. Because it is this way for each stock, we can compute the return as a dot product of returns and weights, then multiply by 250 trading days in a year to annualize our result!

In [ ]:
#operationalize determining portfolio returns
def getPfReturn(weights):
    """
    return is annualized expected return of portfolio
    """
    expRetPortfolio = np.dot(np.transpose(weights), returns_df.mean()) * 250
    return expRetPortfolio

Now, we need to start bounding the function that will maximize our returns given an investment. We will use scipy's minimize function to do this. Despite its name, the minimize function works by minimizing the constraints that you give it, so minimizing the difference between our portfolio return and our target return will enable us to hit our target return, "maximizing" the output of our portfolio. If we want to truly maximize, we can continue pushing up this target return incrementally until we reach it!.

First, we will start with initial weights of each stock in our portfolio. We can either use the weights we selected previously, or we can simply start with each stock having an equal weight in our portfolio. You can choose this by slightly modifying the code below. ###need to add this modification

Now, we need to set our target return value, which we selefcted to arbitrarily be .4, or a 40% return. This can be modified however you wish, but do note that the minimize function will fail if maximizing to the given return percentage is impossible.

Another important goal is to bound the weight of each stock in our portfolio between 0 and 1, or 0% and 100%. It doesn't make much sense to be able to buy more than a full portfolio of stocks! We do this by creating a tuple that has the same length as our number of stocks, and each element in the tuple is another tuple storing the values (0,1), bounding our stock weights in that range

Finally, we want to create two constraints for the function. This is done using the syntax below. Our first constraint says tat the sum of all weights should not exceed 1, or that our portfolio weights can't exceed 100%. We've done this already for individual stocks, but it's also important that we cant just spend 100% of our money in each stock. Finally, we also need to make sure that we "minimize" the difference between the returns for any set of portfolio weights the function generates and the target return we have set. 

In [ ]:
# start with stocks at equal weights
numStocks = len(returns_df.columns)
initialWeight = [1/numStocks] * numStocks

# return goal?

#TODO: figure out what we should set this value to

targetReturn = .4

# bounds the percentage of each stock we can hold (between 0 and 100%)
bounds = tuple((0,1) for i in range(numStocks))

# ensures the sum of all stock weights is 100% (or 1) in first constraint
# sets goal of minimize function to hit targetReturn
constraints = ({'type' : 'eq', 'fun' : lambda w : np.sum(w) - 1},
               {'type' : 'eq', 'fun' : lambda x : x.dot(returns_df.mean()) * 250 - targetReturn})


Now that we have set all of our constraints, we need to use the maximize function! We set a variable called "results" equal to the minimize function with our function and all of the constraints. Through magic of scipy, it will iterate on our weights, modifying them in an attempt to maximize the return of our portfolio. We can print results to see whether it was successful, as well as the weights of each stock in our portfolio. We can also see our returns by using our returns function on the 'x' field of our minimize output.

In [ ]:
# can we reach our goal with these stocks??
results = minimize(fun=getPfReturn, x0=initialWeight, bounds=bounds, constraints=constraints)

#output
print(results)

optimizedResults = pd.DataFrame(results['x'])

# print our optimized results
getPfReturn(weights=results["x"])
optimizedResults.index = returns_df.columns
print(optimizedResults)

Now, we've successfully used scipy to maximize the results from our function! We can even extract the weights and use them to model a portfolio. Let's use some similar code to graph the returns of a portfolio based on an inital investment above, but using our new optimized weights!

In [ ]:
initial_portfolio_val = 1e6

optimizedResults.rename(index={optimizedResults.index[0]: portfolio[0]}, inplace=True)
optimized_allocations = optimizedResults[0].to_dict()


returns = calculate_returns(initial_portfolio_val, dataframes, optimized_allocations)

graph_values(returns, False, True)

### rechecking the strength of your portfolio : isabel's

go back to pyfolio--> will integrate after we recheck the working code